In [263]:
# Importamos las librerías necesarias
from xlsxwriter.utility import xl_rowcol_to_cell, xl_cell_to_rowcol
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as ExcelImage  # Renombramos Image para evitar conflicto con PIL
from PIL import Image as PILImage  # Renombramos también Image de PIL
import pandas as pd
import xlsxwriter
import pathlib
import os

In [272]:
# Definir rutas
ruta_master = os.path.join(str(os.path.abspath(pathlib.Path().absolute())))
ruta_parametros = os.path.join(ruta_master, "Insumo", "Parametros.xlsx")
ruta_json = os.path.join(ruta_master, "Config", "Config.json")
ruta_log = os.path.join(ruta_master, "Log", "Eventos.log")
ruta_resultado = os.path.join(ruta_master, "Resultado")

# Rutas imagenes
logo_alimentos = os.path.join(ruta_master, "util", "Logo alimentos.png")
logo_operador = os.path.join(ruta_master, "util", "Logo operador.png")
logo_secretaria = os.path.join(ruta_master, "util", "Logo secretaria.png")
logo_min_educacion = os.path.join(ruta_master, "util", "Logo Min Educacion.png")

# Rutas archivos
ruta_archivo_aplicacion_novedades = "Insumo\\Focalizacion_actualizada.xlsx"

In [265]:
# Cargamos los parametros
df_parametros = pd.read_excel(ruta_parametros)

# Convertir a diccionario
dict_data = dict(zip(df_parametros["Concepto"], df_parametros["Valor"]))

# Cargamos los parametros por variables
departamento = dict_data["Departamento"]
municipio = dict_data["Municipio"]
operador = dict_data["Operador"]
contrato = dict_data["Contrato No."]
codigo_dane = dict_data["Codigo dane"]
codigo_dane_completo = dict_data["Codigo dane completo"]
jornada = ""
institucion = ""
dane_institucion = ""
mes_atencion = dict_data["Mes de atencion"]
anio = dict_data["Año"]

In [266]:
# Crear un DataFrame vacío
df = pd.DataFrame(index=range(10), columns=[chr(58 + i) for i in range(8)])

# Guardar el DataFrame en un archivo Excel
archivo_excel = "certificado.xlsx"
df.to_excel(archivo_excel, index=False, engine='xlsxwriter')

# Crear una conexión con el archivo Excel y agregar las imágenes
writer = pd.ExcelWriter(archivo_excel, engine='xlsxwriter')
df.to_excel(writer, index=False, header=False, sheet_name='Sheet1')

# Acceder al objeto workbook y worksheet
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Insertar la primera imagen en A3
worksheet.insert_image('A3', logo_alimentos, {'x_scale': 0.3, 'y_scale': 0.3, 'x_offset': 8, 'y_offset': 0})

# Insertar la segunda imagen en A3, desplazándola un poco a la derecha
worksheet.insert_image('A3', logo_min_educacion, {'x_scale': 0.25, 'y_scale': 0.25, 'x_offset': 115, 'y_offset': 8})

# Insertar las imágenes
worksheet.insert_image('C3', logo_operador, {'x_scale': 0.4, 'y_scale': 0.4})
worksheet.insert_image('B3', logo_secretaria, {'x_scale': 0.4, 'y_scale': 0.4})

# Combinar celdas A2 a C5
worksheet.merge_range('A2:C5', '')

# Crear un solo formato reutilizable
formato_celda_unicos = workbook.add_format({
    'bold': True,
    'align': 'left',
    'valign': 'vcenter',
    'font_name': 'Aptos Narrow',
    'font_size': 12,
    'border': 1
})

# Crear un solo formato reutilizable
formato_celda_variables = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Aptos Narrow',
    'font_size': 11,
    'font_color': '#808080',  # Color de la letra en gris
    'border': 1
})

# Crear un solo formato reutilizable
formato_celda_variables_negra = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Aptos Narrow',
    'font_size': 12,
    'border': 1
})

# Combinar celdas D2 a H5 y agregar el texto en negrita
worksheet.merge_range('D2:H5', 'CERTIFICADO DE ENTREGA DE RACIONES A INSTITUCIONES EDUCATIVAS:', 
                        workbook.add_format({
                            'bold': True,
                            'align': 'center',
                            'valign': 'vcenter',
                            'font_name': 'Aptos Narrow',   # Establecer la fuente como Aptos Narrow
                            'font_size': 16
                        }))

# Combinar celdas de A7 a H7
worksheet.merge_range('A7:H7', 'DATOS GENERALES', 
                        workbook.add_format({
                        'bold': True,
                        'align': 'center',
                        'valign': 'vcenter',
                        'font_name': 'Aptos Narrow',   # Establecer la fuente como Aptos Narrow
                        'font_size': 12,        # Tamaño de fuente
                        'bg_color': '#BFBFBF'   # Color de fondo
                    }))

# Aplicar el formato a las celdas
worksheet.write('A8', 'OPERADOR', formato_celda_unicos)
worksheet.write('F8', 'CONTRATO N°:', formato_celda_unicos)
worksheet.write('A10', 'INSTITUCIÓN O CENTRO EDUCATIVO', formato_celda_unicos)
worksheet.write('F10', 'CÓDIGO DANE', formato_celda_unicos)
worksheet.write('A11', 'DEPARTAMENTO:', formato_celda_unicos)
worksheet.write('F11', 'CÓDIGO DANE', formato_celda_unicos)
worksheet.write('A12', 'MUNICIPIO', formato_celda_unicos)
worksheet.write('F12', 'CÓDIGO DANE', formato_celda_unicos)
worksheet.write('A13', 'FECHA DE EJECUCIÓN', formato_celda_unicos)
worksheet.write('B13', 'Desde', formato_celda_unicos)
worksheet.write('E13', 'Hasta', formato_celda_unicos)
worksheet.write('A14', 'NOMBRE RECTOR:', formato_celda_unicos)

# Aplicar el formato a las celdas
worksheet.merge_range('B8:E8', operador, formato_celda_variables_negra)
worksheet.merge_range('G8:H8', contrato, formato_celda_variables_negra)
worksheet.merge_range('G11:H11', codigo_dane, formato_celda_variables)
worksheet.merge_range('B11:E11', departamento, formato_celda_variables)
worksheet.merge_range('G12:H12', codigo_dane_completo, formato_celda_variables)
worksheet.merge_range('B12:E12', municipio, formato_celda_variables)

#############################################################################
# Logica para ingreso de variables 
#############################################################################

# Definir el formato con borde inferior negro
borde_inferior_negro = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'font_size': 11,
    'border': 1,  # Borde inferior negro
})

# Combinar celdas B10 y E10 y agregar el texto de la variable "departamento"
worksheet.merge_range('B10:E10', departamento, borde_inferior_negro)

0

In [ ]:
# Combinar celdas de A17 a H17
worksheet.merge_range('A17:H17', 'CERTIFICACIÓN', 
                        workbook.add_format({
                        'bold': True,
                        'align': 'center',
                        'valign': 'vcenter',
                        'font_name': 'Aptos Narrow',   # Establecer la fuente como Aptos Narrow
                        'font_size': 12,        # Tamaño de fuente
                        'bg_color': '#BFBFBF'   # Color de fondo
                    }))

# Combinar celdas de A18 a H20
worksheet.merge_range('A18:H20', 'El suscrito Rector de la Institución Educativa citada en el encabezado, certifica que se entregaron las siguientes raciones, en las fechas señaladas y de acuerdo con la siguiente distribución:', 
                        workbook.add_format({
                        'align': 'left',
                        'valign': 'vcenter',
                        'font_name': 'Aptos Narrow',   # Establecer la fuente como Aptos Narrow
                        'font_size': 12,         # Tamaño de fuente
                        'border': 1
                    }))

# Definir formato para las celdas combinadas
merge_format = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Aptos Narrow',
    'font_size': 11,
    'bg_color': '#BFBFBF',
    'text_wrap': True
})

# Definir los valores y los rangos a combinar
merge_ranges = {
    'A23:A24': 'NOMBRE DEL ESTABLECIMIENTO EDUCATIVO O CENTRO EDUCATIVO',
    'B23:B24': 'TIPO RACIÓN',
    'F24:H24': 'NOVEDADES'
}

# Aplicar la combinación de celdas con el formato
for rango, texto in merge_ranges.items():
    worksheet.merge_range(rango, texto, merge_format)

# Combinar celdas de C23 a H23
worksheet.merge_range('C23:H23', 'ENTREGADO', 
                        workbook.add_format({
                        'align': 'center',
                        'valign': 'vcenter',
                        'font_name': 'Aptos Narrow',   # Establecer la fuente como Aptos Narrow
                        'font_size': 12,         # Tamaño de fuente
                        'bg_color': '#BFBFBF'    # Color de fondo
                    }))

# Crear un solo formato reutilizable
formato_celda_gris = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Aptos Narrow',
    'font_size': 11,
    'bg_color': '#BFBFBF',    # Color de fondo
    'border': 1
})

worksheet.write('C24', 'N° RACIONES POR DÍA', formato_celda_gris)
worksheet.write('D24', 'N° DÍAS ATENDIDOS', formato_celda_gris)
worksheet.write('E24', 'TOTAL RACIONES', formato_celda_gris)

0

In [275]:
df_focalizacion = pd.read_excel(ruta_archivo_aplicacion_novedades)

df_agrupado = df_focalizacion.groupby(["INSTITUCION", "SEDE"]).size().reset_index(name="TOTAL_REGISTROS")


In [276]:
df_agrupado

,INSTITUCION,SEDE,TOTAL_REGISTROS
0,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,CONCENTRACION RURAL EL CERRITO,93
1,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,CONCENTRACION URBANA EL HATO,412
2,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,CONCENTRACION URBANA MEXICO,166
3,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,CONCENTRACION URBANA SERREZUELITA,185
4,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,CONCENTRACION URBANA SAMARKANDA,148
5,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,FURATENA,949
6,INSTITUCION EDUCATIVA DEPARTAMENTAL DE FUNZA,I.E.D. FUNZA - SEDE PRINCIPAL,794
7,INSTITUCION EDUCATIVA DEPARTAMENTAL TECNICO AG...,CONCENTRACION URBANA SAN MARCOS,835
8,INSTITUCION EDUCATIVA DEPARTAMENTAL TECNICO AG...,INST EDUCATIVA DEPTAL TECNICO AGROPECUARIA SAN...,813
9,INSTITUCION EDUCATIVA TECNICA BICENTENARIO,CONCENTRACION URBANA SAN JOSE,350


In [268]:
# Lista de celdas combinadas para evitar sobrescribir su formato
celdas_combinadas = [
    'D2:H5',  # Ejemplo de combinación
    'A7:H7',
    'B8:E8',
    'G8:H8',
    'A17:H17',
    'A23:A24',
    'B23:B24',
    'C23:H23',
    'F24:H24',
    'A2:C5'
]

# Definir formato con bordes
border_format_combined = workbook.add_format({
    'border': 1,       # Borde en todas las direcciones
    'bold': True,      # Negrita (opcional, si ya lo usaste en otras celdas)
    'align': 'center',  # Alinear al centro
    'valign': 'vcenter' # Alinear verticalmente al centro
})

# Aplicar SOLO el formato con bordes a celdas ya combinadas
for merge_range in celdas_combinadas:
    worksheet.conditional_format(merge_range, {'type': 'no_errors', 'format': border_format_combined})

In [269]:
# 1. Desactivar la cuadrícula
worksheet.hide_gridlines(2)  # 2 es para ocultar la cuadrícula en la vista de diseño

# 2. Rellenar toda la hoja con color blanco
formato_blanco = workbook.add_format({'bg_color': 'white'})
worksheet.set_column('A:Z', None, formato_blanco)  # Rellenar celdas de la A a la Z con color blanco (ajustar según el número de columnas)

0

In [270]:
# 3. Ajustar el tamaño de las columnas de acuerdo con un ancho específico
column_widths = {
    'A': 36.42,  # Ancho de la columna A
    'B': 22.75,  # Ancho de la columna B
    'C': 22.75,  # Ancho de la columna C
    'D': 22.75,  # Ancho de la columna D
    'E': 19.42,  # Ancho de la columna E
    'F': 15.17,  # Ancho de la columna F
    'G': 13.42,  # Ancho de la columna G
    'H': 14,     # Ancho de la columna H
}

# Asignar el ancho especificado a cada columna
for col, width in column_widths.items():
    worksheet.set_column(f'{col}:{col}', width)  # Establecer el ancho para cada columna


In [271]:
# Guardar el archivo con las modificaciones
writer.close()